In [16]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import itertools

import warnings
warnings.filterwarnings('ignore')

In [17]:
#General imports needed for path
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [24]:
#Packages created by our group:
import route_dynamics.route_elevation.base_df as base
import route_dynamics.route_energy.knn as knn
from route_dynamics.route_riders import route_riders as ride
import route_dynamics.route_energy.longi_dynam_model as ldm

In [19]:
#User defines what routes they want to evaluate
rt_list = [22,101,102,143,150,153,154,156,157,158,159,168,169,177,178,179,180,181,182,183,186,187,190,192]

#Import route shapefile
shapefile_name = '../data/rt' + str(rt_list[0]) + '_pts.shp'

#Import bus stops shapefile
routes_shp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'

stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

signals = '../data/Traffic_Signals.shp'

streets = '../data/Metro_Transportation_Network_(TNET)_in_King_County_for_Car_Mode___trans_network_car_line.shp'

trip183 = pd.read_csv("../data/Trip183.csv", usecols = ['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats', 
                     'Period', 'AnnRides']) # KCM Data
trip183unsum = pd.read_csv("../data/Zon183Unsum.csv", usecols = ['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs'])

#Acceleration profile
data = pd.read_csv("../data/acceleration.csv", names=['time (s)', 'accel. (g)'])

In [21]:
rt = gpd.read_file(shapefile_name)

In [23]:
rt.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
data = gpd.read_file(streets)
routes = gpd.read_file(routes_shp)

In [14]:
data.head()

,OBJECTID,TLINK_ID,CREATEDATE,EFFEC_DATE,DB_MD_DATE,END_DATE,KC_FCC_ID,CFCC_ID,HSS,UFLAG,...,A_NAME_R,JURIS_L,JURIS_R,CITY_L,CITY_R,ZIP_L,ZIP_R,ROLL_LEN,Shape_Leng,geometry
0,1,9537,1993-06-28,1993-06-28,2002-01-25,4000-01-01,L,A40,0.0,None,...,None,35,35,Kent,Kent,98031.0,98031.0,342.410418,342.410418,"LINESTRING (-122.21422 47.41899, -122.21424 47..."
1,2,113165,1995-09-28,1995-09-28,2018-01-18,4000-01-01,L,A40,0.0,None,...,None,65,65,Shoreline,Shoreline,98155.0,98155.0,153.300000,153.300073,"LINESTRING (-122.30659 47.77427, -122.30597 47..."
2,3,21455,1993-06-28,1993-06-28,2010-07-14,4000-01-01,C,A40,0.0,None,...,None,76,76,Tukwila,Tukwila,98188.0,98188.0,65.265000,64.742908,"LINESTRING (-122.27349 47.45626, -122.27323 47..."
3,4,173848,2006-10-17,2006-10-17,2017-09-26,4000-01-01,F,A63,0.0,None,...,None,8,8,Bothell,Bothell,98011.0,98011.0,1100.800000,1100.800334,"LINESTRING (-122.18814 47.76591, -122.18854 47..."
4,5,9889,1993-06-28,1993-06-28,1995-11-15,4000-01-01,C,A40,0.0,None,...,None,36,36,King County,King County,98059.0,98059.0,268.068564,268.068564,"LINESTRING (-122.10365 47.48038, -122.10364 47..."


In [6]:
routes_interest = routes.loc[routes['ROUTE_NUM'].isin(rt_list)]
routes_interest = gpd.GeoDataFrame(routes_interest, geometry=routes_interest.geometry)

In [7]:
routes_interest

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry
3,4,140,9,IN SERVICE,Y,100003,L,101,133635.072181,"MULTILINESTRING ((-122.21243 47.47169, -122.21..."
25,26,140,9,IN SERVICE,Y,100041,L,143,119891.223007,"MULTILINESTRING ((-122.00362 47.30946, -122.00..."
30,31,140,9,IN SERVICE,Y,100041,E,143,240699.579584,"MULTILINESTRING ((-122.00362 47.30946, -122.00..."
31,32,140,9,IN SERVICE,Y,100045,L,150,206284.740128,"MULTILINESTRING ((-122.23237 47.38377, -122.23..."
32,33,140,9,IN SERVICE,Y,100048,L,153,66213.643109,"MULTILINESTRING ((-122.23280 47.38459, -122.23..."
...,...,...,...,...,...,...,...,...,...,...
537,538,141,0,NEXT,N,100073,L,182,56357.707206,"MULTILINESTRING ((-122.34453 47.28179, -122.34..."
600,601,141,0,NEXT,N,100489,L,156,91157.048553,"MULTILINESTRING ((-122.30561 47.38658, -122.30..."
602,603,141,0,NEXT,N,100487,L,102,192450.837604,"MULTILINESTRING ((-122.13186 47.44022, -122.13..."
632,633,141,0,NEXT,N,100469,L,180,164136.575415,"MULTILINESTRING ((-122.21913 47.27563, -122.22..."


In [9]:
join = routes_interest.sjoin(data)
join.head()

,OBJECTID_left,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry,...,A_NAME_L,A_NAME_R,JURIS_L,JURIS_R,CITY_L,CITY_R,ZIP_L,ZIP_R,ROLL_LEN,Shape_Leng
3,4,140,9,IN SERVICE,Y,100003,L,101,133635.072181,"MULTILINESTRING ((-122.21243 47.47169, -122.21...",...,None,None,59,59,Renton,Renton,98057.0,98057.0,522.003,522.00329
30,31,140,9,IN SERVICE,Y,100041,E,143,240699.579584,"MULTILINESTRING ((-122.00362 47.30946, -122.00...",...,None,None,59,59,Renton,Renton,98057.0,98057.0,522.003,522.00329
40,41,140,9,IN SERVICE,Y,100061,L,169,89108.372934,"MULTILINESTRING ((-122.20242 47.37256, -122.20...",...,None,None,59,59,Renton,Renton,98057.0,98057.0,522.003,522.00329
158,159,140,9,IN SERVICE,Y,100487,L,102,197619.306927,"MULTILINESTRING ((-122.13186 47.44022, -122.13...",...,None,None,59,59,Renton,Renton,98057.0,98057.0,522.003,522.00329
218,219,140,10,CURRENT,N,100041,E,143,240699.579584,"MULTILINESTRING ((-122.00362 47.30946, -122.00...",...,None,None,59,59,Renton,Renton,98057.0,98057.0,522.003,522.00329


In [ ]:
# m = join.explore()

# routes_interest.explore(m=m,
#               color = 'red')
# folium.TileLayer('Stamen Toner', control=True).add_to(m)  # use folium to add alternative tiles
# folium.LayerControl().add_to(m)  # use folium to add layer control

# m


In [ ]:
join.plot()

In [10]:
route_22 = join.loc[join['ROUTE_NUM'].isin([22])]

In [11]:
route_22 = route_22.iloc[3]

In [12]:
route_22 = route_22.drop(columns=['OBJECTID_left', 'CHANGE_NUM', 'MINOR_CHAN','GRADE', 'A_NAME_L', 'A_NAME_R', 'JURIS_L', 'JURIS_R', 'CITY_L',
'CITY_R', 'ZIP_L', 'ZIP_R', 'index_right', 'OBJECTID_right', 'TLINK_ID', 'PREFIX_R', 'AP_NAME_L', 'AP_NAME_R', 'CREATEDATE','EFFEC_DATE', 'DB_MD_DATE', 'END_DATE', 'KC_FCC_ID', 'CFCC_ID','HSS', 'UFLAG', 'CAR_FLOW'])

In [13]:
route_22

OBJECTID_left                                                    69
CHANGE_NUM                                                      140
MINOR_CHAN                                                        9
CURRENT_NE                                               IN SERVICE
IN_SERVICE                                                        Y
ROUTE_ID                                                     100111
LOCAL_EXPR                                                        L
ROUTE_NUM                                                        22
SHAPE_Leng                                             63300.050133
geometry          (LINESTRING (-122.36591275913531 47.5173631094...
index_right                                                    7212
OBJECTID_right                                                 7213
TLINK_ID                                                       5888
CREATEDATE                                               1993-06-28
EFFEC_DATE                                      

In [ ]:
part_1 = route_22.geometry[0].coords

In [ ]:
part_2 = route_22.geometry[1].coords

In [ ]:
part_3 = route_22.geometry[2].coords

In [ ]:
part_4 = route_22.geometry[3].coords

In [ ]:
part_5 = route_22.geometry[4].coords

In [ ]:
part_6 = route_22.geometry[5].coords

In [ ]:
part_7 = route_22.geometry[6].coords

In [ ]:
part_8 = route_22.geometry[7].coords

In [ ]:
part_9 = route_22.geometry[8].coords

In [ ]:
part_10 = route_22.geometry[9].coords

In [ ]:
part_11 = route_22.geometry[10].coords

In [ ]:
part_12 = route_22.geometry[11].coords

In [ ]:
part_13 = route_22.geometry[12].coords

In [ ]:
part_14 = route_22.geometry[13].coords

In [ ]:
part_15 = route_22.geometry[14].coords

In [ ]:
part_16 = route_22.geometry[15].coords

In [ ]:
part_17 = route_22.geometry[16].coords

In [ ]:
part_18 = route_22.geometry[17].coords

In [ ]:
part_19 = route_22.geometry[18].coords

In [ ]:
part_20 = route_22.geometry[19].coords

In [ ]:
part_21 = route_22.geometry[20].coords

In [ ]:
part_22 = route_22.geometry[21].coords

In [ ]:
part_23 = route_22.geometry[22].coords

In [ ]:
part_24 = route_22.geometry[23].coords

In [ ]:
part_25 = route_22.geometry[24].coords

In [ ]:
list(route_22.geometry[43].coords)

In [ ]:
route_df = base.create_gdf(shapefile_name, 6)

In [ ]:
route_df.head()

In [ ]:
route_22_pts = route_df.sjoin(route_22)

In [ ]:
route_22_pts.head()

In [ ]:
base.create_gdf(shapefile_name, 6)

In [ ]:
df_22, riders_22, mass_22 = ride.route_ridership('PM', 22)

In [ ]:
stop_coord, rider_coord = ride.stop_coord(22, riders_22)

In [ ]:
stop_coords = stop_coord.coordinates.values

In [ ]:
signals = gpd.read_file(stops_shp)

In [ ]:
signal_coords = signals.geometry.values

In [ ]:
event_coords = np.concatenate((stop_coords,signal_coords))

In [ ]:
stop_nn_indicies, stop_coord_nn = knn.find_knn(
1,
route_df.geometry.values,
event_coords
)
# the 'jth' element of stop_nn_indicies also selects the

route_df = route_df.assign(
is_stop = ([False] * len(route_df.index))
)

for i in stop_nn_indicies.ravel():
    route_df.at[i, 'is_stop'] = True

In [ ]:
route_df.is_stop.value_counts()

In [ ]:
#rounding values
data['time (s)'] = data['time (s)'].round(0)
data['accel. (g)'] = data['accel. (g)'].round(4)


In [ ]:
#Convert to SI units
data['accel. (m/s^2)'] = data['accel. (g)']*9.81

In [ ]:
#Calculate Velocity

data['vel. (m/s)'] = np.zeros(len(data.index))

for i in range(1, len(data)):
    data['vel. (m/s)'][i] = ((data['accel. (m/s^2)'][i] + data['accel. (m/s^2)'][i-1])/2*1) + data['vel. (m/s)'][i-1]
    
data['vel. (mph)'] = data['vel. (m/s)']*2.2

In [ ]:
#Calculate Distance

data['dist. (m)'] = np.zeros(len(data.index))

for i in range(1, len(data)):
    data['dist. (m)'][i] = ((data['vel. (m/s)'][i] + data['vel. (m/s)'][i-1])/2*1) + data['dist. (m)'][i-1]

In [ ]:
#Create new dataframe where dist. is the indep var

data2 = pd.DataFrame({'dist. (ft)': np.arange(0, 1260, 36)})
data2['dist. (m)'] = data2['dist. (ft)']/3.28

data2['vel. (m/s)'] = np.zeros(len(data2.index))

data2['vel. (mph)'] = np.zeros(len(data2.index))

data2['accel. (m/s^2)'] = np.zeros(len(data2.index))

data2['time (s)'] = np.zeros(len(data2.index))

In [ ]:
z = data2['dist. (m)'].iloc[2]
params = data.iloc[(data['dist. (m)']-z).abs().argsort()[:1]]
row = params.index.values

#linear interpolation
vel = ((data['vel. (m/s)'][row].values - data['vel. (m/s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['vel. (m/s)'][row-1].values


accel = ((data['accel. (m/s^2)'][row].values - data['accel. (m/s^2)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['accel. (m/s^2)'][row-1].values


In [ ]:
for i in range(1, len(data2)):

    z = data2['dist. (m)'].iloc[i]
    params = data.iloc[(data['dist. (m)']-z).abs().argsort()[:1]]
    row = params.index.values


    #Interpolate
    if params['dist. (m)'].values < z:
        vel = ((data['vel. (m/s)'][row+1].values - data['vel. (m/s)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['vel. (m/s)'][row].values
        accel = ((data['accel. (m/s^2)'][row+1].values - data['accel. (m/s^2)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['accel. (m/s^2)'][row].values
        time = ((data['time (s)'][row+1].values - data['time (s)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['time (s)'][row].values
    else:
        vel = ((data['vel. (m/s)'][row].values - data['vel. (m/s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['vel. (m/s)'][row-1].values
        accel = ((data['accel. (m/s^2)'][row].values - data['accel. (m/s^2)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['accel. (m/s^2)'][row-1].values
        time = ((data['time (s)'][row].values - data['time (s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['time (s)'][row-1].values
        
    
    data2['vel. (m/s)'].iloc[i] = vel
    data2['vel. (mph)'].iloc[i] = vel*2.2
    data2['accel. (m/s^2)'].iloc[i] = accel
    data2['time (s)'].iloc[i] = time

In [ ]:
v_lim = 15 #m/s or ~30 mph
a_neg = -0.4

# Define cutoff distance for acceleration and deceleration
a_avg = data2['accel. (m/s^2)'].mean()
a_plt = data2['accel. (m/s^2)'].iloc[len(data2)-1]
x_a = v_lim**2. / (2*a_avg)
x_d = v_lim**2. / (2*a_neg)

# Distance of accel profile

x_p = data2['dist. (m)'].iloc[34]


In [ ]:
x_ns = np.zeros(len(route_df.index)) #next stops
x_ls = np.zeros(len(route_df.index)) # prev. stops

for i in range(len(x_ns)):
    # set values to Nan if bus stop
    if route_df.at[i, 'is_stop']:
        x_ns[i] = 0.
        x_ls[i] = 0.
        # move to next point
        continue
    else:
        # Calculate 'x_ns';
        # Iterate through remaining indicies to count distance to
        # next stop.
        for j in range(i+1, len(x_ns)):
            # add distance to next point to 'x_ns'
            x_ns[i] += 10.973
            if route_df.at[j, 'is_stop']:
                break # done calulating 'x_ns' at this point
            # elif not bus stop: move to next point, add distance

        # Calculate 'x_ls';
        # Iterate through previous indicies to cout distance to
        # last stop.
        for j in range(i, 0, -1):
            # Inclusive start to range because distances are
            # backward difference. Dont need to include 'j=0'
            # because the first point has no backward difference.
            if route_df.at[j, 'is_stop']:
                break # done calulating x_ls at this point
            x_ls[i] += 10.973


In [ ]:
df = route_df

In [ ]:
v = np.zeros(len(df.index)) #array for vel.
a = np.zeros(len(df.index)) #array for accel.

count = 0

for i in range(len(x_ns)):

    if count > i:
        continue

    else:

        #Case 1

        if (
            x_ns[i]<=abs(x_d)
            and
            not df.at[i, 'is_stop']
            ):
            
            if x_ns[i] < x_ls[i]:

                a[i] = a_neg
                v[i] = np.sqrt(-2*x_ns[i]*a_neg)
                count += 1
                
            else:
                for j in range(len(data2)):

                    if count < len(x_ns):

                        if v[count-1] < v_lim:
                            a[count] = data2['accel. (m/s^2)'].iloc[j]
                            v[count] = data2['vel. (m/s)'].iloc[j]
                            count+=1

                        elif v[count-1]>v_lim:

                            while x_ns[count]>abs(x_d):

                                a[count] = 0
                                v[count] = v_lim

                                count+=1

                            else:
                                continue

                        else:
                            continue

                    else:
                        break


        #Case 2

        elif (
            x_ns[i] > abs(x_d)
            and
            x_ls[i] >= x_p
            and
            not df.at[i, 'is_stop']
            ):

            if v[i-1] < v_lim:

                a[i] = a_plt
                v[i] = np.sqrt(2*x_ns[i]*a[i])
                count += 1

            elif v[i-1]>v_lim:
                        
                while x_ns[count]>abs(x_d):

                    a[count] = 0
                    v[count] = v_lim

                    count+=1

                else:
                    continue 

            else:
                continue

                    

        #Case 3

        elif (
            x_ls[i] < x_p
            and
            x_ns[i] > abs(x_d)
            and
            not df.at[i, 'is_stop']
            ):

            for j in range(len(data2)):

                if count < len(x_ns):

                    if v[count-1] < v_lim:
                        a[count] = data2['accel. (m/s^2)'].iloc[j]
                        v[count] = data2['vel. (m/s)'].iloc[j]
                        count+=1

                    elif v[count-1]>v_lim:
                        
                        while x_ns[count]>abs(x_d):
                
                            a[count] = 0
                            v[count] = v_lim

                            count+=1
                            
                        else:
                            continue
                            
                    else:
                        continue

                else:
                    break


        elif df.at[i, 'is_stop']:

            count += 1

In [ ]:
df['velocity'] = v

In [ ]:
df['accel'] = a

In [ ]:
back_diff_delta_x = np.full(len(df), 10.973)

velocities = df.velocity.values

segment_avg_velocities = (
                velocities
                +
                np.append(0,velocities[:-1])
                )/2

df['delta_times'] = back_diff_delta_x / segment_avg_velocities
delta_times = df['delta_times'].values
delta_times[delta_times > 1000000] = 0

In [ ]:
time_on_route = np.append(0, np.cumsum(delta_times[1:]))
df['time'] = time_on_route
df['time(min)'] = df['time']/60

In [ ]:
df.head()

In [ ]:
def calc_forces(df):

    # Physical parameters
    gravi_accel = 9.81
    air_density = 1.225 # air density in kg/m3; consant for now,
        # eventaully input from weather API
    v_wind = 0.0 # wind speed in km per hour; figure out component,
        # and also will come from weather API
    fric_coeff = 0.01

    # List of Bus Parameters for 40 foot bus

    loaded_bus_mass = df.mass.values

    width = 2.6 # in m
    height = 3.3 # in m
    bus_front_area = width * height
    drag_coeff = 0.34 # drag coefficient estimate from paper (???)
    rw = 0.28575 # radius of wheel in m
    
    vels = df.velocity.values
    acce = df.accel.values
    grad = df.grade.values
    grad_angle = np.arctan(grad)
    
    # Calculate the gravitational force
    grav_force = -(
        loaded_bus_mass * gravi_accel * np.sin(grad_angle)
        )

    # Calculate the rolling friction
    roll_fric = -(
        fric_coeff * loaded_bus_mass * gravi_accel * np.cos(grad_angle)
        )

    # Calculate the aerodynamic drag
    aero_drag = -(
        drag_coeff
        *
        bus_front_area
        *
        (air_density/2)
        *
        (vels-v_wind)
        )

    # Calculate the inertial force
    inertia = loaded_bus_mass * acce

    return (grav_force, roll_fric, aero_drag, inertia)
